In [2]:
import torch as pt
import torch.nn as nn
from torch.utils.data import random_split
from src.dataset import MolDataset
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn

/home/dima/miniconda3/envs/ctpesto/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.hid = nn.Linear(hidden_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.hid(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [28]:
pt.manual_seed(110)

In [2]:
device = "cuda" if pt.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [29]:
N = 18000
dataset = MolDataset("../data/preprocessed/X.pk", 
                     y_datafile="../data/preprocessed/Y.pk", normal = True)
train_dataset, test_dataset = random_split(dataset, [len(dataset) - N, N])

In [30]:
test_dataset[:10][1]

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [77]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(42, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.hid = nn.Linear(hidden_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.hid(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [78]:

model = NeuralNet(42,512,3).to(device)
print(model)

NeuralNet(
  (fc1): Linear(in_features=42, out_features=512, bias=True)
  (relu): ReLU()
  (hid): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)


In [79]:
X1 = np.array(train_dataset[81][0])
X1 = X1[None, :]
X = pt.tensor(X1, device=device)
# X = torch.unsqueeze(X, dim=0)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([[0, 0, 0]], device='cuda:0')


In [80]:
class_weights = pt.Tensor([18, 18, 1])
class_weights.to(device)

tensor([18., 18.,  1.], device='cuda:0')

In [81]:
# class_weights = [0.8, 0.8, 0.1]
loss_fn = nn.BCEWithLogitsLoss(weight=class_weights)
# loss_fn = nn.CrossEntropyLoss(weight=pt.tensor([5.,5.,1.]))
optimizer = pt.optim.Adam(model.parameters())

In [82]:
loss_fn.to(device)

BCEWithLogitsLoss()

In [83]:

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        # Compute prediction and loss
        x = X.to(device)
        y = Y[:,None,:].to(device)
        pred = model(x)
        # print(pred,y)
        loss = loss_fn(pred, y.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 200 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with pt.no_grad():
        for X, Y in dataloader:
            x = X.to(device)
            y = Y[:,None,:].to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y.float()).item()
    #             correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    #     correct /= size
    #     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Avg loss: {test_loss:>8f} \n")



In [85]:
epochs = 7
# train model
train_dataloader = DataLoader(train_dataset, batch_size=1024,
                              shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=1024,
                             shuffle=True, pin_memory=True)

for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    # logger.store_progress(0, is_train=True, epoch=t+1)
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.492551  [    0/52711]
Avg loss: 1.877420 

Epoch 2
-------------------------------
loss: 1.688777  [    0/52711]
Avg loss: 1.882305 

Epoch 3
-------------------------------
loss: 1.919940  [    0/52711]
Avg loss: 1.899627 

Epoch 4
-------------------------------
loss: 1.538989  [    0/52711]
Avg loss: 1.894449 

Epoch 5
-------------------------------
loss: 1.572258  [    0/52711]
Avg loss: 1.897527 

Epoch 6
-------------------------------
loss: 1.590208  [    0/52711]
Avg loss: 1.908379 

Epoch 7
-------------------------------
loss: 1.755535  [    0/52711]
Avg loss: 1.901678 

Done!


In [70]:
pt.save(model.state_dict(), "model.pt")

In [86]:
with pt.no_grad():
    for X, Y in train_dataloader:
        x = X.to(device)
        y = Y[:,None,:].to(device)

In [87]:
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

In [88]:
x.shape

torch.Size([487, 1, 42])

In [90]:
logits[0:10]

tensor([[[-4.3805, -3.5956,  3.1753]],

        [[-3.0419, -3.5770,  2.5254]],

        [[-2.3859, -2.6075,  1.9775]],

        [[-3.8351, -2.7911,  2.4693]],

        [[-4.3270, -3.1107,  2.8222]],

        [[-2.8488, -3.5158,  2.5445]],

        [[-3.3458, -2.9673,  2.3939]],

        [[-4.0330, -3.0436,  2.6762]],

        [[-4.4319, -3.2398,  2.9119]],

        [[-3.8627, -3.2036,  2.6865]]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [75]:
y[0:8,:,:]

tensor([[[0., 0., 1.]],

        [[0., 0., 1.]],

        [[0., 0., 1.]],

        [[0., 0., 1.]],

        [[0., 0., 1.]],

        [[1., 0., 0.]],

        [[0., 1., 0.]],

        [[0., 0., 1.]]], device='cuda:0', dtype=torch.float64)